## Задание 2. SQL

### 2.1 Очень усердные ученики.

#### 2.1.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.



#### 2.1.2 Задача

Дана таблица `default.peas`:

Название атрибута | Тип атрибута | Смысловое значение
------------------|--------------|-------------------
st_id | int | ID ученика
timest | timestamp | Время решения карточки
correct | bool | Правильно ли решена горошина?
subject | text | Дисциплина, в которой находится горошина

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

### 2.2 Оптимизация воронки

#### 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

#### 2.2.2 Задача


Дана таблицы: `default.peas` (см. выше), `default.studs`:

Название атрибута | Тип атрибута |Смысловое значение
------------------|--------------|-------------------
st_id|int |ID ученика
test_grp|text |Метка ученика в данном эксперименте

и `default.final_project_check`


Название атрибута|Тип атрибута |Смысловое значение
------------------|--------------|-------------------
st_id|int |ID ученика
sale_time|timestamp|Время покупки
money|int|Цена, по которой приобрели данный курс
subject|text |Дисциплина, на которую приобрели полный доступ


Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

* ARPU
* ARPAU
* CR в покупку
* СR активного пользователя в покупку
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
* ARPU считается относительно всех пользователей, попавших в группы.

**Активным** считается пользователь, за все время решивший **больше** 10 задач правильно в любых дисциплинах.

**Активным** по математике считается пользователь, за все время решивший **2 или больше** задач правильно по математике.

*Все данные находятся в табличном виде в ClickHouse*

### Задание 1

Для работы с `clickhouse` нам понадобиться установить а потом импортировать библиотеку `pandahouse`.

In [ ]:
!pip install pandahouse -q

  Preparing metadata (setup.py) ... done


In [ ]:
import pandahouse as ph
import pandas as pd

In [ ]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student',
                      'password':'dpo_python_2020'
                     }

Напишем небольшую функцию для работы с запросами.

In [ ]:
def select(sql: str):
    '''returns pandas dataframe from clickhouse'''
    return ph.read_clickhouse(sql, connection=connection_default)

Посмотрим за какой период у нас представлены данные.

In [ ]:
sql = '''SELECT min(toDateTime(timest)) as min_date,
max(toDateTime(timest)) as max_date
FROM default.peas'''

In [ ]:
select(sql)

,min_date,max_date
0,2021-10-30 13:00:00,2021-10-31 17:29:52


Мы видим что в таблице данные только за один день, поэтому нам не нужно фильтровать данные по месяцу.

In [ ]:
sql = '''SELECT count(distinct(st_id)) as uniq_students
FROM(
    SELECT st_id, sum(correct) as correct
    FROM peas
    GROUP BY st_id
    HAVING correct >= 20
    )
'''

In [ ]:
print(f"Количество очень усердных студентов: {select(sql)['uniq_students'].values[0]}")

Количество очень усердных студентов: 136


### Задание 2

In [ ]:
query_1 = '''
select
        test_grp as group,
        sum(revenue) / count(peas.st_id) as ARPU,
        sumIf(revenue, correct_answer > 10) / countIf(peas.st_id, correct_answer > 10) as ARPAU,
        (countIf(peas.st_id, revenue > 0) / count(peas.st_id)) * 100 as CR,
        (countIf(peas.st_id, (revenue > 0 and correct_answer > 10)) / countIf(peas.st_id, correct_answer > 10)) * 100 as CR_active_users,
        (countIf(peas.st_id, (revenue_math > 0 and active_math >= 2)) / countIf(peas.st_id, active_math >= 2)) * 100 as CR_active_math
from default.studs as studs
left join
    (select st_id, sum(money) as revenue,
    sumIf(money, subject = 'Math') as revenue_math
    from default.final_project_check
    group by st_id)
    as check
on studs.st_id = check.st_id
left join
    (select st_id, sum(correct) as correct_answer,
    sumIf(correct, subject = 'Math') as active_math
    from default.peas
    group by st_id) as peas
    on studs.st_id = peas.st_id
group by test_grp'''

In [ ]:
select(query_1)

,group,ARPU,ARPAU,CR,CR_active_users,CR_active_math
0,control,4540.983607,10393.700787,4.918033,11.023622,6.122449
1,pilot,11508.474576,29739.583333,10.847458,26.041667,9.523810


Все метрики в тестовой группе выросли, нам нужно проверить статзначимость различий в метриках и после можно будет делать вывод о целесообразности введения нового экрана оплаты.